In [33]:
from ch2.data import data
from ch2.lib.date import to_time
d = data()
print(d.activity_groups())
a = d.activity_journals('Bike')
x = a.loc[(a.index >= to_time('2017-01-18')) & (a.index < to_time('2017-01-19'))].iloc[0]
print(x)
ride = d.statistic_journals('Spherical Mercator X', 'Spherical Mercator Y', 'Latitude', 'Longitude', 'Distance', source_id=x.source_id)
print(ride)

    INFO: Using database at /home/andrew/.ch2/database.sqlm
INFO:no-op:Using database at /home/andrew/.ch2/database.sqlm
INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
INFO:sqlalchemy.engine.base.Engine:()
INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
INFO:sqlalchemy.engine.base.Engine:()
INFO:sqlalchemy.engine.base.Engine:PRAGMA table_info("source")
INFO:sqlalchemy.engine.base.Engine:()
INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.base.Engine:SELECT activity_group.id AS activity_group_id, activity_group.name AS activity_group_name, activity_group.description AS activity_group_description, activity_group.sort AS activity_group_sort 
FROM activity_group ORDER BY activity_group.name
INFO:sqlalchemy.engine.base.Engine:()
INFO:sqlalchemy.engine.base.Engine:SELECT count(activity_journal.id) AS count_1 
FROM source JOIN activity_journal ON source.id = activity_journa

   name             description  count
1  Bike  All cycling activities    334
2   Run  All running activities      0
name                                           2017-01-18-pm
fit_file     /home/andrew/archive/fit/bike/2017-01-18-pm.fit
finish                             2017-01-18 11:34:37+00:00
source_id                                              33583
Name: 2017-01-18 10:39:33+00:00, dtype: object
                            Latitude  Longitude  Spherical Mercator X  \
2017-01-18 10:39:33+00:00 -33.427251 -70.608018         -7.860049e+06   
2017-01-18 10:39:34+00:00 -33.427253 -70.608019         -7.860049e+06   
2017-01-18 10:39:50+00:00 -33.427280 -70.608014         -7.860048e+06   
2017-01-18 10:39:55+00:00 -33.427306 -70.607912         -7.860037e+06   
2017-01-18 10:39:57+00:00 -33.427303 -70.607881         -7.860033e+06   
2017-01-18 10:40:02+00:00 -33.427322 -70.607858         -7.860031e+06   
2017-01-18 10:40:04+00:00 -33.427312 -70.607853         -7.860030e+06   
2017-01-

In [36]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row
from bokeh.models.widgets import Slider, PreText

output_notebook()
width, height = 800, 800

def modify_doc(doc):

    t1 = PreText(height=20, width=200)
    t2 = PreText(height=20, width=200)
    t3 = PreText(height=20, width=100)
    s1 = Slider(start=0, end=len(ride), value=0, title='Start')
    s2 = Slider(start=1, end=len(ride)-1, value=len(ride), title='Length')
    f = figure(plot_width=width, plot_height=height, x_axis_type='mercator', y_axis_type='mercator')
    c = column(row(s1, s2), row(t1, t2, t3), f)

    def mkplot(l1, l2):
        l2 = min(len(ride)-1, l1+l2)
        t1.text = '%9.5f,%9.5f' % (ride.iloc[l1]['Longitude'], ride.iloc[l1]['Latitude'])
        t2.text = '%9.5f,%9.5f' % (ride.iloc[l2]['Longitude'], ride.iloc[l2]['Latitude'])
        t3.text = '%4.1fkm' % ((ride.iloc[l2]['Distance'] - ride.iloc[l1]['Distance']) / 1000)
        f = figure(plot_width=width, plot_height=height, x_axis_type='mercator', y_axis_type='mercator')
        f.circle(x='Spherical Mercator X', y='Spherical Mercator Y', source=ride[l1:l2])
        f.add_tile(CARTODBPOSITRON)
        c.children[2] = f

    s1.on_change('value', lambda attr, old, new: mkplot(s1.value, s2.value))
    s2.on_change('value', lambda attr, old, new: mkplot(s1.value, s2.value))
    doc.add_root(c)

show(modify_doc)


Loading BokehJS ...

In [42]:
from ch2.squeal.database import connect
from ch2.squeal.tables.segment import Segment
ns, log, db = connect(['-v', '5'])
with db.session_context() as s:
    x = Segment(start=(-70.61813,-33.41536), finish=(-70.63340,-33.42655), distance=4400,
               name='San Cristobal', description='Climb up San Cristobal in Parque Metropolitana')
    s.add(x)

    INFO: Using database at /home/andrew/.ch2/database.sqlm
INFO:no-op:Using database at /home/andrew/.ch2/database.sqlm
INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
INFO:sqlalchemy.engine.base.Engine:()
INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
INFO:sqlalchemy.engine.base.Engine:()
INFO:sqlalchemy.engine.base.Engine:PRAGMA table_info("source")
INFO:sqlalchemy.engine.base.Engine:()
INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.base.Engine:INSERT INTO segment (start_lat, start_lon, finish_lat, finish_lon, distance, name, description) VALUES (?, ?, ?, ?, ?, ?, ?)
INFO:sqlalchemy.engine.base.Engine:(-33.41536, -70.61813, -33.42655, -70.6334, 4400.0, 'San Cristobal', 'Climb up San Cristobal in Parque Metropolitana')
INFO:sqlalchemy.engine.base.Engine:COMMIT
